In [9]:
!echo %cd% # under windows
#!pwd # under linux/mac

%cd%


In [10]:
import pandas as pd
import numpy as np
import csv
import string
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore') 
from IPython.display import clear_output
from sklearn.model_selection import train_test_split


## Import Data

**Wikigold**

- Labels: 'I-MISC', 'O', 'I-ORG', 'I-PER', 'I-LOC'
- Split into train/test set by taking first 80% of words for train and last 20% of words for test (to preserve sentences)

**Twitter (train/test)**

- Labels: 'O', 'B-location', 'I-location', 'B-group', 'B-corporation', 'B-person', 'B-creative-work', 'B-product', 'I-person', 'I-creative-work', 'I-corporation', 'I-group', 'I-product'
- Map labels to match wikigold, both I/B tags are mapped to I
- Remove punctuation

**Financial SEC-filings (train/test)**

- Labels: 'O' 'I-ORG' 'I-LOC' 'I-PER' 'I-MISC'
- Remove punctuation

**Anatomical AnEM (train/test)**

- Labels: 'B-Multi-tissue_structure', 'O', 'B-Organism_substance', 'B-Organism_subdivision', 'B-Organ', 'I-Multi-tissue_structure', 'B-Cellular_component', 'I-Cellular_component', 'B-Cell', 'I-Cell', 'B-Immaterial_anatomical_entity', 'B-Tissue', 'I-Tissue',
'B-Pathological_formation', 'B-Anatomical_system', 'I-Organism_substance', 'I-Anatomical_system', 'I-Pathological_formation', 'I-Immaterial_anatomical_entity', 'I-Organ', 'I-Organism_subdivision', 'B-Developing_anatomical_structure', 'I-Developing_anatomical_structure'
- Remove punctuation


**B- denotes the beginning and I- inside of an entity. The prefixes are used to detect multiword entities, as shown in the example example above. All other words, which don’t refer to entities of interest, are labelled with the O tag.**

## Import Data

### 1. Wikigold

In [11]:
#wiki = pd.read_csv('C:/Users/Phil/Sync/entity-recognition-datasets-master/data/wikigold/CONLL-format/data/wikigold.conll.txt', sep=" ", header=None)
# wiki = pd.read_csv('~/Sync/entity-recognition-datasets-master/data/wikigold/CONLL-format/data/wikigold.conll.txt', sep="\n| ", header=None)
wiki = pd.read_csv('C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data/wikigold/CONLL-format/data/wikigold.conll.txt', sep="\n| ", header=None)   
wiki.columns = ['word','label']

FileNotFoundError: [Errno 2] No such file or directory: 'C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data/wikigold/CONLL-format/data/wikigold.conll.txt'

In [ ]:
wiki.head()

In [ ]:
print(wiki['label'].unique())

# First remove puncutation and rows that only have punctuation, e.g. "&"
wiki_2 = wiki.copy()
wiki_2['word'] = wiki_2['word'].str.extract('(\w+)', expand = False)
wiki_2 = wiki_2.dropna().reset_index(drop=True)
wiki_2.head()

In [ ]:
#wiki['label'] = np.where(wiki['label']=='O','O','tag')

In [ ]:
wiki_train = wiki_2.iloc[:int(len(wiki_2)*0.8),:]
wiki_test = wiki_2.iloc[int(len(wiki_2)*0.8):,:].reset_index(drop=True)

### 2. Twitter WNUT17

In [ ]:
#social_media_train = pd.read_csv('C:/Users/Phil/Sync/entity-recognition-datasets-master/data/WNUT17/CONLL-format/data/train/wnut17train.conll', sep="\t", header=None)
# social_media_train = pd.read_csv('~/Sync/entity-recognition-datasets-master/data/WNUT17/CONLL-format/data/train/wnut17train.conll', sep="\t", header=None)
social_media_train = pd.read_csv('C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data/WNUT17/CONLL-format/data/train/wnut17train.conll', sep="\t", header=None)
    
social_media_train.columns = ['word','label']
print(social_media_train['label'].unique())

In [ ]:
# Map labels to those consistent from wiki
"""
social_media_train['label'] = np.where(social_media_train['label']=='I-location', 'I-LOC',
                              np.where(social_media_train['label']=='I-person', 'I-PER',
                              np.where(social_media_train['label']=='I-corporation', 'I-ORG',
                              np.where(social_media_train['label']=='I-group', 'I-ORG', 
                              np.where(social_media_train['label']=='I-creative-work', 'I-MISC',
                              np.where(social_media_train['label']=='I-product', 'I-MISC', 'O'))))))
"""
social_media_train['label'] = np.where((social_media_train['label']=='I-location') | (social_media_train['label']=='B-location'), 'I-LOC',
                              np.where((social_media_train['label']=='I-person') | (social_media_train['label']=='B-person'), 'I-PER',
                              np.where((social_media_train['label']=='I-corporation') | (social_media_train['label']=='B-corporation'), 'I-ORG',
                              np.where((social_media_train['label']=='I-group') | (social_media_train['label']=='B-group'), 'I-ORG', 
                              np.where((social_media_train['label']=='I-creative-work') | (social_media_train['label']=='B-creative-work'), 'I-MISC',
                              np.where((social_media_train['label']=='I-product') | (social_media_train['label']=='B-product'), 'I-MISC', 'O'))))))





#social_media_train['label'] = np.where(social_media_train['label']=='O','O','tag')





# First remove puncutation and rows that only have punctuation, e.g. "&"
social_media_train_2 = social_media_train.copy()
social_media_train_2['word'] = social_media_train_2['word'].str.extract('(\w+)', expand = False)
social_media_train_2 = social_media_train_2.dropna().reset_index(drop=True)
social_media_train_2.head()

In [ ]:
# social_media_test = pd.read_csv('~/Sync/entity-recognition-datasets-master/data/WNUT17/CONLL-format/data/test/emerging.test.annotated', sep="\t", header=None, quoting=csv.QUOTE_NONE, encoding='utf-8')
social_media_test = pd.read_csv("C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data/WNUT17/CONLL-format/data/test/emerging.test.annotated", sep="\t", header=None, quoting=csv.QUOTE_NONE, encoding='utf-8')  

social_media_test.columns = ['word','label']
print(social_media_test['label'].unique())

In [ ]:
# Map labels to those consistent from wiki
"""
social_media_test['label'] = np.where(social_media_test['label']=='I-location', 'I-LOC',
                              np.where(social_media_test['label']=='I-person', 'I-PER',
                              np.where(social_media_test['label']=='I-corporation', 'I-ORG',
                              np.where(social_media_test['label']=='I-group', 'I-ORG', 
                              np.where(social_media_test['label']=='I-creative-work', 'I-MISC',
                              np.where(social_media_test['label']=='I-product', 'I-MISC', 'O'))))))
"""

social_media_test['label'] = np.where((social_media_test['label']=='I-location') | (social_media_test['label']=='B-location'), 'I-LOC',
                              np.where((social_media_test['label']=='I-person') | (social_media_test['label']=='B-person'), 'I-PER',
                              np.where((social_media_test['label']=='I-corporation') | (social_media_test['label']=='B-corporation'), 'I-ORG',
                              np.where((social_media_test['label']=='I-group') | (social_media_test['label']=='B-group'), 'I-ORG', 
                              np.where((social_media_test['label']=='I-creative-work') | (social_media_test['label']=='B-creative-work'), 'I-MISC',
                              np.where((social_media_test['label']=='I-product') | (social_media_test['label']=='B-product'), 'I-MISC', 'O'))))))



#social_media_test['label'] = np.where(social_media_test['label']=='O','O','tag')


# First remove puncutation and rows that only have punctuation, e.g. "&"
social_media_test_2 = social_media_test.copy()
social_media_test_2['word'] = social_media_test_2['word'].str.extract('(\w+)', expand = False)
social_media_test_2 = social_media_test_2.dropna().reset_index(drop=True)
social_media_test_2.head()

### 3. Finance SEC-filings	

In [ ]:
#finance_train = pd.read_csv('C:/Users/Phil/Sync/entity-recognition-datasets-master/data/SEC-filings/CONLL-format/data/train/FIN5.txt', sep=" ", header=None)
# finance_train = pd.read_csv('~/Sync/entity-recognition-datasets-master/data/SEC-filings/CONLL-format/data/train/FIN5.txt', sep=" - ", header=None)
finance_train = pd.read_csv("C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data/SEC-filings/CONLL-format/data/train/FIN5.txt", sep=" - ", header=None) 

# finance_train = pd.read_csv('', sep=" - ", header=None)

finance_train = finance_train.reset_index()
finance_train.columns = ['word_tag','label']

finance_train.replace(to_replace=[None], value=np.nan, inplace=True)
finance_train = finance_train.dropna()
finance_train = finance_train.reset_index(drop=True)

word_tag_list = pd.DataFrame()
for n,word_tag in enumerate(finance_train['word_tag']):
    word = word_tag.split(' ')[0]
    tag = word_tag.split(' ')[1]
    word_tag_list = word_tag_list.append(pd.DataFrame({'word':word,
                                                      'tag':tag}, index =[n]))
    
finance_train['word'] = word_tag_list['word']
finance_train['tag'] = word_tag_list['tag']
finance_train = finance_train.iloc[:,[2,1]]

print(finance_train['label'].unique())


# First remove puncutation and rows that only have punctuation, e.g. "&"
finance_train_2 = finance_train.copy()
finance_train_2['word'] = finance_train_2['word'].str.extract('(\w+)', expand = False)
finance_train_2 = finance_train_2.dropna().reset_index(drop=True)


In [ ]:
#finance_train['label'] = np.where(finance_train['label']=='O','O','tag')

In [ ]:
#finance_test = pd.read_csv('C:/Users/Phil/Sync/entity-recognition-datasets-master/data/SEC-filings/CONLL-format/data/test/FIN3.txt', sep=" ", header=None)
# finance_test = pd.read_csv('~/Sync/entity-recognition-datasets-master/data/SEC-filings/CONLL-format/data/test/FIN3.txt', sep=" - ", header=None)
finance_test = pd.read_csv('C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data/SEC-filings/CONLL-format/data/test/FIN3.txt', sep=" - ", header=None)


finance_test = finance_test.reset_index()
finance_test.columns = ['word_tag','label']

finance_test.replace(to_replace=[None], value=np.nan, inplace=True)
finance_test = finance_test.dropna()
finance_test = finance_test.reset_index(drop=True)

word_tag_list = pd.DataFrame()
for n,word_tag in enumerate(finance_test['word_tag']):
    word = word_tag.split(' ')[0]
    tag = word_tag.split(' ')[1]
    word_tag_list = word_tag_list.append(pd.DataFrame({'word':word,
                                                      'tag':tag}, index =[n]))
    
finance_test['word'] = word_tag_list['word']
finance_test['tag'] = word_tag_list['tag']
finance_test = finance_test.iloc[:,[2,1]]

print(finance_test['label'].unique())
# First remove puncutation and rows that only have punctuation, e.g. "&"
finance_test_2 = finance_test.copy()
finance_test_2['word'] = finance_test_2['word'].str.extract('(\w+)', expand = False)
finance_test_2 = finance_test_2.dropna().reset_index(drop=True)


### 4. Anatomical AnEM (labels do not map)

In [ ]:
#anem_train = pd.read_csv('C:/Users/Phil/Sync/entity-recognition-datasets-master/data/AnEM/CONLL-format/data/AnEM.train', sep="\t", header=None)
# anem_train = pd.read_csv('~/Sync/entity-recognition-datasets-master/data/AnEM/CONLL-format/data/AnEM.train', sep="\t", header=None)
anem_train = pd.read_csv('C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data/AnEM/CONLL-format/data/AnEM.train', sep="\t", header=None)
# C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data

anem_train = anem_train.iloc[:,[0,3]]
anem_train.columns = ['word','label']

# First remove puncutation and rows that only have punctuation, e.g. "&"
anem_train_2 = anem_train.copy()
anem_train_2['word'] = anem_train_2['word'].str.extract('(\w+)', expand = False)
anem_train_2 = anem_train_2.dropna().reset_index(drop=True)

print(anem_train_2['label'].unique())

In [ ]:
#anem_train_2['label'] = np.where(anem_train_2['label']=='O','O','tag')

In [ ]:
#anem_test = pd.read_csv('C:/Users/Phil/Sync/entity-recognition-datasets-master/data/AnEM/CONLL-format/data/AnEM.test', sep="\t", header=None)
# anem_test = pd.read_csv('~/Sync/entity-recognition-datasets-master/data/AnEM/CONLL-format/data/AnEM.test', sep="\t", header=None)
anem_test = pd.read_csv('C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data/AnEM/CONLL-format/data/AnEM.test', sep="\t", header=None)

# C:/data/Projects/Manchester_Uni_of/Trasportability/BERT_tutorial/Data/entity-recognition-datasets-master/data
anem_test = anem_test.iloc[:,[0,3]]
anem_test.columns = ['word','label']

# First remove puncutation and rows that only have punctuation, e.g. "&"
anem_test_2 = anem_test.copy()
anem_test_2['word'] = anem_test_2['word'].str.extract('(\w+)', expand = False)
anem_test_2 = anem_test_2.dropna().reset_index(drop=True)

print(anem_test_2['label'].unique())

In [ ]:
#anem_test_2['label'] = np.where(anem_test_2['label']=='O','O','tag')

## NER Pipeline 

**Inputs:** 

1. Source_x,
2. Source_y,
3. Target_x, 
4. Target_y

**Training:**

1. Build a *Dictionary Vectorizer* on Source_x
2. Extract labels from Source_y
3. Split Source_x & Source_y into an internal train/test split randomly (80/20%)
4. Fit a Multinomial NB model (alpha = 0.01) to this internal train set
5. Evaluate this on the internal test set for the **Training Results (precision/recall/F1)**

**Application to Target data**

1. Compute **Feature Gradient Measure** as proportion of unique words from Target_x that appear in training Source_x
2. Fit Target_x to Source_x's *Dictionary Vectorizer*
3. Apply trained NB model to this
4. Evaluate **Applied Results** **(precision/recall/F1)**

In [ ]:
def NB_NER_Pipeline_2(source_x, source_y, source_name, target_x, target_y, target_name, num_samples, model):
    import sklearn
    from sklearn.feature_extraction import DictVectorizer
    from sklearn.feature_extraction.text import HashingVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import classification_report
    from sklearn.linear_model import Perceptron

    # Establish training classes
    X_source = source_x
    v_source = DictVectorizer(sparse=False)
    X_source = v_source.fit_transform(X_source.to_dict('records'))
    y_source = source_y.values
    
    classes_source = np.unique(y_source)
    classes_source = classes_source.tolist()
    
    if model == 'NB':
        # NB classifier trained on source
        X_train_source, X_test_source, y_train_source, y_test_source = train_test_split(X_source, y_source, test_size = 0.2, random_state=0)
        nb_source = MultinomialNB(alpha=0.01)
        nb_source.partial_fit(X_train_source, y_train_source, classes_source)
        y_pred_source = nb_source.predict(X_test_source)
        output_source = pd.DataFrame({'pred':y_pred_source, 'actual':y_test_source})
        precision_training = sklearn.metrics.precision_score(output_source['actual'],output_source['pred'], average='weighted')
        recall_training = sklearn.metrics.recall_score(output_source['actual'],output_source['pred'], average='weighted')   
        F1_training = 2 * (precision_training * recall_training) / (precision_training + recall_training)
    
    elif model=='Perceptron':
    
        # Perceptron classifier trained on source
        X_train_source, X_test_source, y_train_source, y_test_source = train_test_split(X_source, y_source, test_size = 0.2, random_state=0)
        per_source = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
        per_source.partial_fit(X_train_source, y_train_source, classes_source)
        y_pred_source = per_source.predict(X_test_source)
        output_source = pd.DataFrame({'pred':y_pred_source, 'actual':y_test_source})
        precision_training = sklearn.metrics.precision_score(output_source[output_source['actual']!='O']['actual'],
                                                             output_source[output_source['actual']!='O']['pred'], average='weighted')
        recall_training = sklearn.metrics.recall_score(output_source[output_source['actual']!='O']['actual'],output_source[output_source['actual']!='O']['pred'], average='weighted')   
        F1_training = 2 * (precision_training * recall_training) / (precision_training + recall_training)
    
    
    source_x['counter'] = 1
    source_x['label'] = source_y
    # Sampling target data by num_samples
    sampling_output = pd.DataFrame()
    for sample in range(0, num_samples):
        target_x_sample = target_x.iloc[int((sample/num_samples)*len(target_x))  : int(((sample+1)/num_samples)*len(target_x)),:]
        target_y_sample = target_y.iloc[int((sample/num_samples)*len(target_x))  : int(((sample+1)/num_samples)*len(target_x))]
    
        # Compute measure
        target_x_sample_measure = target_x_sample
        target_x_sample_measure['label'] = target_y_sample
        target_x_sample_measure = target_x_sample_measure[target_x_sample_measure['label']!='O'].merge(source_x, how='left', on=['word','label'])#.drop_duplicates(['word','label']) #SHOULD WE COMPUTER WITHOUT 'O' TAGS???
        sample_measure = 1-(target_x_sample_measure['counter'].isna().sum()/len(target_x_sample_measure))
    
        # Source to Target model
        X_source_target = v_source.transform(target_x_sample.to_dict('records'))
        y_target_actual = target_y_sample
        
        if model == 'NB':
            y_source_target_pred=nb_source.predict(X_source_target)
        elif model=='Perceptron':
            y_source_target_pred=per_source.predict(X_source_target)

        
        output_source_target = pd.DataFrame({'pred':y_source_target_pred, 'actual':y_target_actual})
        # Fix applied results to be training results if source is same as target
        if source_name == target_name:
            precision_applied = precision_training
            recall_applied = recall_training
        else:
            precision_applied = sklearn.metrics.precision_score(output_source_target[output_source_target['actual']!='O']['actual'],
                                                                output_source_target[output_source_target['actual']!='O']['pred'], average='weighted')
            recall_applied = sklearn.metrics.recall_score(output_source_target[output_source_target['actual']!='O']['actual'],
                                                      output_source_target[output_source_target['actual']!='O']['pred'], average='weighted')
        F1_applied = 2 * (precision_applied * recall_applied) / (precision_applied + recall_applied)

        sampling_output = sampling_output.append(pd.DataFrame({
                                                               'Source':source_name,
                                                               'Target':target_name,
                                                               'sample':(sample+1),
                                                               'measure':sample_measure,
                                                               'train_prec':precision_training,
                                                               'train_rec':recall_training,
                                                               'train_F1':F1_training,
                                                               'applied_prec':precision_applied,
                                                               'applied_rec':recall_applied,
                                                               'applied_F1':F1_applied,
                                                               'adjusted_F1':(F1_applied)/F1_training
                                                               }, index=[sample]))

    return(sampling_output)

In [ ]:
source_x = wiki.drop('label', axis=1)
source_y = wiki['label']
source_name = 'Wiki'


target_x = social_media_test_2.drop('label', axis=1)
target_y = social_media_test_2['label']
target_name = 'Social Media: test'

num_samples = 50

model = 'NB'

Mdl2 = NB_NER_Pipeline_2(source_x, source_y, source_name, target_x, target_y, target_name, num_samples, model)

In [ ]:
source_x

In [ ]:
Mdl2.head()

In [ ]:
plt.scatter(Mdl2['measure'],Mdl2['applied_F1'])
plt.title(model+' Transportability Measure against \n F1 score when Trained on Source applied to Target')
plt.xlabel('Transportabiltiy Measure')
plt.ylabel('Applied Accuracy')
plt.show()

In [ ]:
plt.scatter(Mdl2['measure'],Mdl2['applied_F1'])
plt.title(model+' Transportability Measure against \n F1 score when Trained on Source applied to Target')
plt.xlabel('Transportabiltiy Measure')
plt.ylabel('Applied Accuracy')
plt.ylim([0,0.6])
plt.xlim([0,0.6])
plt.show()

In [ ]:
regression = pd.DataFrame()
regression['measure'] = Mdl2['measure']
regression['applied_F1'] = Mdl2['applied_F1']
regression = regression.dropna()
from scipy.stats import linregress
linregress(regression['measure'],regression['applied_F1'])

In [ ]:
regression = regression.dropna()

## Switching Source/Target Systematically

In [ ]:
datasets = [wiki_train, wiki_test, social_media_train_2, social_media_test_2, finance_train_2, finance_test_2, anem_train_2, anem_test_2]
dataset_names = ['Wikigold_train', 'Wikigold_test', 'Twitter_train', 'Twitter_test', 'SEC-filings_train', 'SEC-filings_test', 'Anem_train', 'Anem_test']

In [ ]:
EDA_output = pd.DataFrame()

for n1,dataset in enumerate(datasets):
    dataset_EDA = pd.DataFrame({'dataset': dataset_names[n1], 'size':len(dataset), 'vocab size':len(dataset['word'].unique()),
                               'vocab ratio':len(dataset['word'].unique())/len(dataset),
                               'NER_words':len(dataset[dataset['label']!='O']['word']) ,
                               'NER_words_unique':len(dataset[dataset['label']!='O']['word'].unique()),
                                
                               'NER_words_ratio':len(dataset[dataset['label']!='O']['word'].unique())/len(dataset['word'].unique())
                               
                               }, index = [n1])
    EDA_output = EDA_output.append(dataset_EDA)

In [ ]:
EDA_output

In [ ]:
plt.bar(EDA_output['dataset'],EDA_output['size'])
plt.title("Size of Datasets")
plt.ylabel("Number of Words")
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.bar(EDA_output['dataset'],EDA_output['vocab size'])
plt.title("Vocab Size of Datasets")
plt.ylabel("Number of Unique Words")
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.bar(EDA_output['dataset'],EDA_output['NER_words'])
plt.title("NER Words in Datasets")
plt.ylabel("Number of NER Words")
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.bar(EDA_output['dataset'],EDA_output['NER_words_unique'])
plt.title("NER Unique Words of Datasets")
plt.ylabel("Number of Unique NER Words")
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
num_samples = 1
model = 'NB'

combined_output = pd.DataFrame()
for n1,source in enumerate(datasets):
    for n2,target in enumerate(datasets):

            source_x = source.drop('label', axis=1)
            source_y = source['label']
            source_name = dataset_names[n1]
            

            target_x = target.drop('label', axis=1)
            target_y = target['label']
            target_name = dataset_names[n2]
            
            clear_output(wait=True)
            print("Source Datasets Progress: ", np.round((n1+1)/len(datasets),4)*100, "%")
            print("Source: ", source_name)
            print("Target Datasets Progress: ", np.round((n2+1)/len(datasets),4)*100, "%")
            print("Target: ", target_name)
            

            Mdl = NB_NER_Pipeline_2(source_x, source_y, source_name, target_x, target_y, target_name, num_samples, model)
            combined_output = combined_output.append(Mdl).reset_index(drop=True)
            


In [ ]:
EDA_output

In [ ]:
combined_output

In [ ]:
combined_output['measure'] = combined_output['measure'].round(decimals=2)
combined_output['train_prec'] = combined_output['train_prec'].round(decimals=2)
combined_output['train_rec'] = combined_output['train_rec'].round(decimals=2)
combined_output['train_F1'] = combined_output['train_F1'].round(decimals=2)
combined_output['applied_prec'] = combined_output['applied_prec'].round(decimals=2)
combined_output['applied_rec'] = combined_output['applied_rec'].round(decimals=2)
combined_output['applied_F1'] = combined_output['applied_F1'].round(decimals=2)
combined_output['adjusted_F1'] = combined_output['adjusted_F1'].round(decimals=2)




In [ ]:
combined_output_piv = combined_output.replace(0,np.nan).pivot("Source", "Target", "measure")
ax = sns.heatmap(combined_output_piv, annot=True, cbar=False, cmap='Greys')
plt.title("Feature Gradient")
ax.invert_yaxis()
ax.invert_xaxis()

In [ ]:
combined_output_piv = combined_output.pivot("Source", "Target", "train_F1")
ax = sns.heatmap(combined_output_piv, annot=True, cbar=False)
plt.title("Training F1 Score")
ax.invert_yaxis()
ax.invert_xaxis()

In [ ]:
combined_output_piv = combined_output.pivot("Source", "Target", "applied_F1")
ax = sns.heatmap(combined_output_piv, annot=True, cbar=False,cmap='Greys')
plt.title("Applied F1 Score")
ax.invert_yaxis()
ax.invert_xaxis()

In [ ]:
combined_output_piv = combined_output.pivot("Source", "Target", "adjusted_F1")
ax = sns.heatmap(combined_output_piv, annot=True, cbar=False, cmap='Greys')
plt.title(r"$\tau_p$" + " F1 Score \n (Applied Score given Training Score)")
ax.invert_yaxis()
ax.invert_xaxis()

In [ ]:
plt.scatter(combined_output[combined_output['Source']!=combined_output['Target']]['measure'],combined_output[combined_output['Source']!=combined_output['Target']]['adjusted_F1'])
plt.title('Correlation of Feature Gradient Measure against ' + r"$\tau_p$")
plt.xlabel('Feature Gradient')
plt.ylim(0,1)
plt.xlim(0,1)
plt.ylabel(r"$\tau_p$" + ' F1 Score')
plt.show()

In [ ]:
regression = pd.DataFrame()
regression['measure'] = combined_output[combined_output['Source']!=combined_output['Target']]['measure']
regression['applied_F1'] = combined_output[combined_output['Source']!=combined_output['Target']]['adjusted_F1']
regression = regression.dropna()
from scipy.stats import linregress
linregress(regression['measure'],regression['applied_F1'])

In [ ]:
plt.scatter(combined_output[(combined_output['measure']!=1)&(combined_output['applied_F1']<0.5)]['measure'],
            combined_output[(combined_output['measure']!=1)&(combined_output['applied_F1']<0.5)]['adjusted_F1'])
plt.title('Correlation of Feature Gradient Measure against ' + r"$\tau_p$")
plt.xlabel('Feature Gradient')
plt.ylim(0,1)
plt.xlim(0,1)
plt.ylabel(r"$\tau_p$" + ' F1 Score')
plt.show()

In [ ]:
regression = pd.DataFrame()
regression['measure'] = combined_output[(combined_output['measure']!=1)&(combined_output['applied_F1']<0.5)]['measure']
regression['applied_F1'] = combined_output[(combined_output['measure']!=1)&(combined_output['applied_F1']<0.5)]['adjusted_F1']
regression = regression.dropna()
from scipy.stats import linregress
linregress(regression['measure'],regression['applied_F1'])

### Robustness Pipeline




**Inputs:** 

1. Source_x,
2. Source_y,
3. Target_x, 
4. Target_y

**Training:**

1. Build a *Dictionary Vectorizer* on Source_x
2. Extract labels from Source_y
3. Split Source_x & Source_y into an internal train/test split randomly (80/20%)
4. Fit a Multinomial NB model (alpha = 0.01) to this internal train set
5. Evaluate this on the internal test set for the **Training Results (precision/recall/F1)**

**Application to Target data**

1. Compute **Feature Gradient Measure** as proportion of unique words from Target_x that appear in training Source_x
2. Fit Target_x to Source_x's *Dictionary Vectorizer*
3. Apply trained NB model to this
4. Evaluate **Applied Results** **(precision/recall/F1)**


$$Robustness = \bigg(1+\frac{1}{4n}\bigg)\frac{\sqrt{\frac{\sum_i(\textrm{perf}(\Psi,T,\mathcal{D}'_i)-\mean{\textrm{perf}(\Psi,T,\mathcal{D})})^2}{n-1}}}{\mean{\textrm{perf}(\Psi,T,\mathcal{D})}}$$

In [ ]:
def robustness(source_x, source_y, source_name, num_samples, model):
    import sklearn
    from sklearn.feature_extraction import DictVectorizer
    from sklearn.feature_extraction.text import HashingVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import classification_report
    from sklearn.linear_model import Perceptron

    # Establish training classes
    X_source = source_x
    v_source = DictVectorizer(sparse=False)
    X_source = v_source.fit_transform(X_source.to_dict('records'))
    y_source = source_y.values
    
    classes_source = np.unique(y_source)
    classes_source = classes_source.tolist()
    
    sample_output = pd.DataFrame()
    for sample in range(0,num_samples):
    
        if model == 'NB':
            # NB classifier trained on source
            X_train_source, X_test_source, y_train_source, y_test_source = train_test_split(X_source, y_source, test_size = 0.2)
            nb_source = MultinomialNB(alpha=0.01)
            nb_source.partial_fit(X_train_source, y_train_source, classes_source)
            y_pred_source = nb_source.predict(X_test_source)
            output_source = pd.DataFrame({'pred':y_pred_source, 'actual':y_test_source})
            precision_training = sklearn.metrics.precision_score(output_source['actual'],output_source['pred'], average='weighted')
            recall_training = sklearn.metrics.recall_score(output_source['actual'],output_source['pred'], average='weighted')   
            F1_training = 2 * (precision_training * recall_training) / (precision_training + recall_training)

        elif model=='Perceptron':

            # Perceptron classifier trained on source
            X_train_source, X_test_source, y_train_source, y_test_source = train_test_split(X_source, y_source, test_size = 0.2)
            per_source = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
            per_source.partial_fit(X_train_source, y_train_source, classes_source)
            y_pred_source = per_source.predict(X_test_source)
            output_source = pd.DataFrame({'pred':y_pred_source, 'actual':y_test_source})
            precision_training = sklearn.metrics.precision_score(output_source[output_source['actual']!='O']['actual'],
                                                                 output_source[output_source['actual']!='O']['pred'], average='weighted')
            recall_training = sklearn.metrics.recall_score(output_source[output_source['actual']!='O']['actual'],output_source[output_source['actual']!='O']['pred'], average='weighted')   
            F1_training = 2 * (precision_training * recall_training) / (precision_training + recall_training)

        sample_output = sample_output.append(pd.DataFrame({'sample':sample, 'F1':F1_training}, index = [sample]))
    mean_F1 = sample_output['F1'].mean()
    sample_output['mean_F1'] = mean_F1
    sample_output['F1_-_mean_F1_squared'] = np.square(sample_output['F1']-sample_output['mean_F1'])
    n = len(sample_output)
    robustness = (1 + (1/(4*n))) * ( np.sqrt( (sample_output['F1_-_mean_F1_squared'].sum())/(n-1)  ) /mean_F1 )
    return(robustness)

In [ ]:
num_samples = 10
model = 'NB'

robustness_output = pd.DataFrame()
for n1,source in enumerate(datasets):
    source_x = source.drop('label', axis=1)
    source_y = source['label']
    source_name = dataset_names[n1]



    clear_output(wait=True)
    print("Source Datasets Progress: ", np.round((n1+1)/len(datasets),4)*100, "%")
    print("Source: ", source_name)


    Mdl = robustness(source_x, source_y, source_name, num_samples, model)
    robustness_output = robustness_output.append(pd.DataFrame({'Source': source_name, 'Robustness':Mdl},index = [n1]))


In [ ]:
robustness_output

In [ ]:
robustness_output['Robustness_norm'] = (robustness_output['Robustness'] - min( robustness_output['Robustness']))/(max( robustness_output['Robustness'])-min( robustness_output['Robustness']))
robustness_output

In [ ]:
plt.bar(robustness_output['Source'], robustness_output['Robustness'])
plt.title("Robustness of Sources with 10 Samples")
plt.xticks(rotation='vertical')
plt.show()

### Context Stability 

$$\textrm{Context Stability } \tau_s = \bigg(1+\frac{1}{4n}\bigg) \frac{\sqrt{\frac{\sum_{i=1}^n(\tau_p(\mathcal{D}_0,\mathcal{D}_i)-\mean{\tau_p})^2}{n-1}}}{\mean{\tau_p}}$$

In [ ]:
context_stability_output = pd.DataFrame()
for n1,source in enumerate(datasets):
    source_name = dataset_names[n1]
    source_results = combined_output[combined_output['Source']==source_name].dropna()
    source_results = source_results[source_results['Source']!=source_results['Target']]
    mean_applied_F1 = source_results['applied_F1'].mean()
    source_results['mean_applied_F1'] = mean_applied_F1
    source_results['applied_F1_-_mean_applied_F1_squared'] = np.square(source_results['applied_F1']-source_results['mean_applied_F1'])
    
    n = len(source_results)
    context_stability = (1 + (1/(4*n))) * ( np.sqrt( (source_results['applied_F1_-_mean_applied_F1_squared'].sum())/(n-1)  ) /mean_applied_F1 )
    context_stability_output = context_stability_output.append(pd.DataFrame({'Source': source_name, 'Context stability':context_stability},index = [n1]))
context_stability_output = context_stability_output.fillna(0)

In [ ]:
context_stability_output

In [ ]:
context_stability_output['Context stability_norm'] = (context_stability_output['Context stability'] - min( context_stability_output['Context stability']))/(max( context_stability_output['Context stability'])-min( context_stability_output['Context stability']))
context_stability_output

In [ ]:
plt.bar(context_stability_output['Source'], context_stability_output['Context stability'])
plt.title("Stability of Sources")
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
combined_output.head()

In [ ]:
combined_output_2 = combined_output.merge(context_stability_output, how='left', on = 'Source')
combined_output_2.head(10)

In [ ]:
combined_output_3 = combined_output_2.merge(robustness_output, how='left', on = 'Source')
combined_output_3.head(10)

In [ ]:
combined_output_3[['Source', 'Target','measure','applied_F1','adjusted_F1','Context stability', 'Robustness']][0:8]

### Word to Vec Mapping Analysis

following guide found here: 

http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/

"Using Google’s pre-trained model. It’s 1.5GB! It includes word vectors for a vocabulary of 3 million words and phrases that they trained on roughly 100 billion words from a Google News dataset. The vector length is 300 features."



In [ ]:
import gensim

In [ ]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('~/Sync/entity-recognition-datasets-master/GoogleW2V/GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
w2v_model['queen'][0:10]

In [ ]:
len(w2v_model['queen'])

In [ ]:
wiki.head()

In [ ]:
w2v_model['is']

In [ ]:
wiki_word_list = pd.DataFrame()
for n,word in enumerate(wiki['word']):
    clear_output(wait=True)
    print("Words Completed: ", np.round(n/len(wiki),4)*100,"%")
    try:
        word_vector = w2v_model[str(word)]
    except:
        word_vector = "missing" #np.zeros(300, dtype=np.float32)
    wiki_word_list = wiki_word_list.append(pd.DataFrame({'vector':[word_vector]}, index = [n]))

In [ ]:
wiki['vector'] = wiki_word_list['vector']
wiki.head(10)

In [ ]:
wiki[wiki['label']!='O'].head(20)

In [ ]:
print("Percentage of words missing in vector lookup = ",(len(wiki[wiki['vector']=="missing"])/len(wiki))*100, "%")

In [ ]:
print("Percentage of NER words missing in vector lookup = ",(len(wiki[(wiki['vector']=="missing")&(wiki['label']!='O')])/len(wiki[wiki['label']!='O']))*100, "%")

To account for these missing lookup words, I will simply set the vector to a list of 0s if missing for now. 

First, I repeat this lookup process for each dataset.

Also need to normalise the values to be positive as NB won't accept negative values.

In [ ]:
print(dataset_names)

In [ ]:
for d,dataset in enumerate(datasets):
    vector_word_list = pd.DataFrame()
    for n,word in enumerate(datasets[d]['word']):
        clear_output(wait=True)
        print("DATASET: ", dataset_names[d])
        print("Dataset: ", np.round(d/len(dataset_names),4)*100, "%")
        print("Words Completed: ", np.round(n/len(datasets[d]),4)*100,"%")
        try:
            word_vector = w2v_model[str(word)]
            #Normalise [-1:1] to [0:1], replace outliers that are less than -1 originally
            #word_vector = (word_vector - (-1))/(1 - (-1))
            #word_vector = [0 if i <0 else i for i in word_vector]
        except:
            word_vector = np.zeros(300, dtype=np.float32)
        vector_word_list = vector_word_list.append(pd.DataFrame({'vector':[word_vector]}, index = [n]))
    
    datasets[d]['vector'] = vector_word_list['vector']

In [ ]:
datasets[0].head()

In [ ]:
def NB_NER_Pipeline_3(source_x, source_y, source_name, target_x, target_y, target_name, num_samples, model):
    import sklearn
    from sklearn.feature_extraction import DictVectorizer
    from sklearn.feature_extraction.text import HashingVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import classification_report
    from sklearn.linear_model import Perceptron

    # Establish training classes
    X_source = np.array(source_x['vector'].tolist())
    # ALREADY TRANSFORMED BY LOOKUP
    #v_source = DictVectorizer(sparse=False)
    #X_source = v_source.fit_transform(X_source.to_dict('records'))
    y_source = source_y.values
    
    classes_source = np.unique(y_source)
    classes_source = classes_source.tolist()
    
    if model == 'NB':
        # NB classifier trained on source
        X_train_source, X_test_source, y_train_source, y_test_source = train_test_split(X_source, y_source, test_size = 0.2, random_state=0)
        nb_source = MultinomialNB(alpha=0.01)
        nb_source.partial_fit(X_train_source, y_train_source, classes_source)
        y_pred_source = nb_source.predict(X_test_source)
        output_source = pd.DataFrame({'pred':y_pred_source, 'actual':y_test_source})
        precision_training = sklearn.metrics.precision_score(output_source['actual'],output_source['pred'], average='weighted')
        recall_training = sklearn.metrics.recall_score(output_source['actual'],output_source['pred'], average='weighted')   
        F1_training = 2 * (precision_training * recall_training) / (precision_training + recall_training)
    
    elif model=='Perceptron':
    
        # Perceptron classifier trained on source
        X_train_source, X_test_source, y_train_source, y_test_source = train_test_split(X_source, y_source, test_size = 0.2, random_state=0)
        per_source = Perceptron(verbose=0, n_jobs=-1, max_iter=1000, alpha=0.0009)
        per_source.partial_fit(X_train_source, y_train_source, classes_source)
        y_pred_source = per_source.predict(X_test_source)
        output_source = pd.DataFrame({'pred':y_pred_source, 'actual':y_test_source})
        precision_training = sklearn.metrics.precision_score(output_source[output_source['actual']!='O']['actual'],
                                                             output_source[output_source['actual']!='O']['pred'], average='weighted')
        recall_training = sklearn.metrics.recall_score(output_source[output_source['actual']!='O']['actual'],output_source[output_source['actual']!='O']['pred'], average='weighted')   
        F1_training = 2 * (precision_training * recall_training) / (precision_training + recall_training)
    
    
    source_x['counter'] = 1
    
    # Sampling target data by num_samples
    sampling_output = pd.DataFrame()
    for sample in range(0, num_samples):
        target_x_sample = target_x.iloc[int((sample/num_samples)*len(target_x))  : int(((sample+1)/num_samples)*len(target_x)),:]
        target_y_sample = target_y.iloc[int((sample/num_samples)*len(target_y))  : int(((sample+1)/num_samples)*len(target_y))]
     
        # Compute measure
        target_x_sample_measure = target_x_sample[target_y_sample!='O'][['word']].merge(source_x[['word','counter']], how='left', on='word').drop_duplicates() #SHOULD WE COMPUTER WITHOUT 'O' TAGS???
        sample_measure = 1-(target_x_sample_measure['counter'].isna().sum()/len(target_x_sample_measure))
    
        # Source to Target model - NOT NEEDED IN AS THIS IS ALREADY PRE-FITTED
        #X_source_target = v_source.transform(target_x_sample.to_dict('records'))
        X_target = np.array(target_x_sample['vector'].tolist())
        y_target_actual = target_y_sample
        
        if model == 'NB':
            y_source_target_pred=nb_source.predict(X_target)
        elif model=='Perceptron':
            y_source_target_pred=per_source.predict(X_target)

        
        output_source_target = pd.DataFrame({'pred':y_source_target_pred, 'actual':y_target_actual})
        # Fix applied results to be training results if source is same as target
        if source_name == target_name:
            precision_applied = precision_training
            recall_applied = recall_training
        else:
            precision_applied = sklearn.metrics.precision_score(output_source_target[output_source_target['actual']!='O']['actual'],
                                                                output_source_target[output_source_target['actual']!='O']['pred'], average='weighted')
            recall_applied = sklearn.metrics.recall_score(output_source_target[output_source_target['actual']!='O']['actual'],
                                                      output_source_target[output_source_target['actual']!='O']['pred'], average='weighted')
       
        F1_applied = 2 * (precision_applied * recall_applied) / (precision_applied + recall_applied)

        sampling_output = sampling_output.append(pd.DataFrame({
                                                               'Source':source_name,
                                                               'Target':target_name,
                                                               'sample':(sample+1),
                                                               'measure':sample_measure,
                                                               'train_prec':precision_training,
                                                               'train_rec':recall_training,
                                                               'train_F1':F1_training,
                                                               'applied_prec':precision_applied,
                                                               'applied_rec':recall_applied,
                                                               'applied_F1':F1_applied,
                                                               'adjusted_F1':(F1_applied)/F1_training
                                                               }, index=[sample]))

    return(sampling_output)

In [ ]:
num_samples = 1
model = 'Perceptron'

combined_output_Vec = pd.DataFrame()
for n1,source in enumerate(datasets):
    for n2,target in enumerate(datasets):

            source_x = source.drop('label', axis=1)
            source_y = source['label']
            source_name = dataset_names[n1]
            

            target_x = target.drop('label', axis=1)
            target_y = target['label']
            target_name = dataset_names[n2]
            
            clear_output(wait=True)
            print("Source Datasets Progress: ", np.round((n1+1)/len(datasets),4)*100, "%")
            print("Source: ", source_name)
            print("Target Datasets Progress: ", np.round((n2+1)/len(datasets),4)*100, "%")
            print("Target: ", target_name)
            

            Mdl = NB_NER_Pipeline_3(source_x, source_y, source_name, target_x, target_y, target_name, num_samples, model)
            combined_output_Vec = combined_output_Vec.append(Mdl).reset_index(drop=True)
            

In [ ]:
combined_output_piv_Vec = combined_output_Vec.pivot("Source", "Target", "applied_F1")
ax = sns.heatmap(combined_output_piv_Vec, annot=True, cbar=False, cmap='BuPu')
plt.title("Applied F1 Score (W2V)")
ax.invert_yaxis()
ax.invert_xaxis()

In [ ]:
combined_output_Vec[combined_output_Vec['Source']=='SEC-filings_train']